# stringdale standard library

Given our diagram's ability to use generic functions, we can create a standard library of functions that are useful for LLMs.

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from stringdale.doc import show_doc

## Condition

In [ ]:
from stringdale.std import Condition

In [ ]:
show_doc(Condition) #TODO redo show doc so it shown return values and signature properly

 ## Condition
<p align="right"> <a href="https://github.com/DeanLight/stringdale/blob/main/stringdale/std.py">source</a> </p>

> **Signature:** `Condition(func: Union[Callable, str], mapping: Optional[str] = None, name: Optional[str] = None)`

A utility function for creating condition functions using stringdale's port mapping logic.


| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| func | typing.Union[typing.Callable, str] | None | A function or string to match against the input. If string, it will be interpreted as a regex pattern to match the input against. |
| mapping | typing.Optional[str] | None | A mapping of the input to the function. by default, the input is the first argument of the function. |
| name | typing.Optional[str] | None | A name for the condition. by default, is func_name->mapping_string |



### \_\_call\_\_

None
> **Signature:** `Condition.__call__(self, x)`

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| x |  |  |  |




### Example

In [ ]:
def is_module_2(x):
    return x%2==0

is_input_even = Condition(is_module_2,'(x=input)',name='input is even')

assert is_input_even({"input": 2})
assert not is_input_even({"input": 3})

In [ ]:
def is_sum_42(x,y):
    return x+y==42

is_io_42 = Condition(is_sum_42,'(x=input,y=output)',name='i+o=42?')

assert is_io_42({"input": 2,'output':40})
# missing inputs results in False
assert not is_io_42({"input": 2,})
assert not is_io_42({"input": 2,'output':39})

In [ ]:
C = Condition

complex_cond = (
    C("^3",'(0=input)',name='starts with 3') & 
    C("^5",'(0=output)',name='starts with 5') | 
    C("^2",'(0=input)',name='starts with 2'))
complex_cond

((starts with 3 & starts with 5) | starts with 2)

In [ ]:
assert complex_cond({"input": "345", "output": "567"})
assert complex_cond({"input": "244", "output": "567"})
assert not complex_cond({"input": "145", "soutput": "567"})
assert not complex_cond({"input": "345", "output": "67"})
assert not complex_cond({"not_input": "145"})
assert complex_cond({ "input": "244"})

## StructureJson

In [ ]:
from stringdale.std import StructureJson

In [ ]:
show_doc(StructureJson)

 ## StructureJson
<p align="right"> <a href="https://github.com/DeanLight/stringdale/blob/main/stringdale/std.py">source</a> </p>

> **Signature:** `StructureJson(*assignments)`

A class for restructuring JSON objects by nested paths
This class allows restructuring JSON objects by specifying path assignments in the form
'target_path=source_path'. It parses these assignments and creates a new JSON object
with the specified structure.

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| assignments | None | None | Variable number of strings specifying path assignments in the format'target_path=source_path' |
| :Returns: | None | - | A new JSON object with the specified structure. |



### \_\_call\_\_

None
> **Signature:** `StructureJson.__call__(self, **kwargs)`

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| kwargs |  |  |  |
| :Returns: | None | - | A new JSON object with the specified structure. |




### Example

In [ ]:
source_obj = {'d':{'e':{'f':'foo'},'x':'bar'}}

StructureJson('a.b.c=father.d.e.f','a.b.d=father.d.x')(father=source_obj)

{'a': {'b': {'c': 'foo', 'd': 'bar'}}}

## JsonRenderer

In [ ]:
from stringdale.std import JsonRenderer

In [ ]:
show_doc(JsonRenderer)

 ## JsonRenderer
<p align="right"> <a href="https://github.com/DeanLight/stringdale/blob/main/stringdale/std.py">source</a> </p>

> **Signature:** `JsonRenderer(json_obj, **kwargs)`

A class for rendering JSON objects with nested jinja2 templates.

Allows setting template variables both during init and when calling the object.

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| json_obj | None | None | A JSON object to render. |
| **kwargs | None | None | Context variables to use in the rendering. |
| kwargs |  |  |  |
| :Returns: | None | - | A rendered JSON object. |



### \_\_call\_\_

None
> **Signature:** `JsonRenderer.__call__(self, **kwargs)`

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| kwargs |  |  |  |
| :Returns: | None | - | A rendered JSON object. |




### Example

In [ ]:
template_json = {
    'name': '{{name}}',
    'age': '{{age}}',
    'city_path': '{%for city in cities%} {{city}} - {%endfor%}'
}

rend = JsonRenderer(template_json,age=30,cities=['SF','LA','NY'])
rend


JsonRenderer(missing_keys={'name'})

In [ ]:
rend(name='Bob')

{'name': 'Bob', 'age': '30', 'city_path': 'SF -  LA -  NY -'}